In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import model_selection

### Training Set

In [2]:
cust_data = pd.read_csv('data_transform.csv')
cust_data.shape

(100233, 279)

In [3]:
cust_data.head()

,ACTL_FMLY_NUM,AGE,ARPU,AUTR_FAIL_MCNT,AUTR_FAIL_YN,AVG_CALL_FREQ,AVG_CALL_TIME,AVG_STLN_RATE,CB_GUIF_AMT,CB_GUIF_CNT,...,MATE_OCCP_NAME_C_07,MATE_OCCP_NAME_C_08,MATE_OCCP_NAME_C_09,MATE_OCCP_NAME_C_10,MATE_OCCP_NAME_C_11,MATE_OCCP_NAME_C_12,MATE_OCCP_NAME_C_13,MATE_OCCP_NAME_C_14,MATE_OCCP_NAME_C_15,MATE_OCCP_NAME_C_16
0,4,50,30000,10,1.0,493,450,0,420001,3,...,0,0,0,0,0,0,0,0,1,0
1,4,50,30000,0,0.0,22,81,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,4,60,30000,0,0.0,17,139,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,35,30000,0,0.0,0,1118,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,4,45,50000,0,0.0,354,396,95,0,0,...,0,0,0,0,0,0,0,0,1,0


### Test Set

In [4]:
test_data = pd.read_csv('test_transform.csv')
test_data = test_data.drop('TARGET', axis=1)
test_data.shape

(2019, 278)

### Score Fuction

In [5]:
def model_performance(y_test, y_pred):    
    print('confusion matrix')
    print(metrics.confusion_matrix(y_test, y_pred))
    print('accuracy : {}'.format(metrics.accuracy_score(y_test, y_pred).round(3)))
    print('precision : {}'.format(metrics.precision_score(y_test, y_pred, pos_label=1).round(3)))
    print('recall : {}'.format(metrics.recall_score(y_test, y_pred, pos_label=1).round(3)))
    print('F1 : {}'.format(metrics.f1_score(y_test, y_pred, pos_label=1).round(3)))

## 연체자 data duplicate

In [6]:
cust = cust_data[cust_data['TARGET'] == 0]
cust_overdue = cust_data[cust_data['TARGET'] == 1]     # 연체자

print(cust.shape[0])
print(cust_overdue.shape[0])

95946
4287


In [7]:
variables = cust_data.drop('TARGET', 1).columns
variables

Index(['ACTL_FMLY_NUM', 'AGE', 'ARPU', 'AUTR_FAIL_MCNT', 'AUTR_FAIL_YN',
       'AVG_CALL_FREQ', 'AVG_CALL_TIME', 'AVG_STLN_RATE', 'CB_GUIF_AMT',
       'CB_GUIF_CNT',
       ...
       'MATE_OCCP_NAME_C_07', 'MATE_OCCP_NAME_C_08', 'MATE_OCCP_NAME_C_09',
       'MATE_OCCP_NAME_C_10', 'MATE_OCCP_NAME_C_11', 'MATE_OCCP_NAME_C_12',
       'MATE_OCCP_NAME_C_13', 'MATE_OCCP_NAME_C_14', 'MATE_OCCP_NAME_C_15',
       'MATE_OCCP_NAME_C_16'],
      dtype='object', length=278)

### ML modeling

In [8]:
# train / test set 분리
x = cust_data.drop('TARGET', axis=1)
y = cust_data['TARGET']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

print('연체자 수 train : {} / {}'.format(sum(y_train == 1), y_train.shape[0]))
print('연체자 수 test  : {} / {}'.format(sum(y_test  == 1), y_test.shape[0]))

연체자 수 train : 3439 / 80186
연체자 수 test  : 848 / 20047


In [9]:
# train set 중에서 연체자만 추출
x_overdue = x_train[y_train == 1]
y_overdue = y_train[y_train == 1]
print(x_overdue.shape)
print(y_overdue.shape)

(3439, 278)
(3439,)


In [10]:
# clf = GradientBoostingClassifier(n_estimators=500, random_state=0)

In [11]:
max_epoch = 2

for i in range(max_epoch+1):
    x_train = pd.concat([x_train, x_overdue])
    y_train = pd.concat([y_train, y_overdue])

    x2_train, x2_test, y2_train, y2_test = train_test_split(x_train, y_train, test_size=0.2, random_state=35228)
#     x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

    print('========================================= ', i)
    print('train overdue {}'.format(sum(y_train == 1)))
    print('')
    
    clf = GradientBoostingClassifier(n_estimators=500, random_state=0)
#     clf = GradientBoostingClassifier(n_estimators=500, max_features='auto', random_state=0)
#     clf = GradientBoostingClassifier(n_estimators=500, max_features='auto', learning_rate=0.01, random_state=0)

    clf.fit(x2_train, y2_train)
#     clf.fit(x_train, y_train)
    
#     varDic = {'var':variables, 'importance':clf.feature_importances_}
#     importance = pd.DataFrame(varDic)
#     print(importance.sort_values(by='importance', ascending=False).head(10))

    print('------------------------')
#     y_pred = clf.predict(x2_test)
#     model_performance(y2_test, y_pred)

    y_pred_0 = clf.predict(x_test)
    model_performance(y_test, y_pred_0)
    print('------------------------')

    if i == max_epoch:
        y_pred_test = clf.predict(test_data)
        test_data['TARGET'] = y_pred_test

=========================================  0
train overdue 6878

------------------------
confusion matrix
[[18891   308]
 [  552   296]]
accuracy : 0.957
precision : 0.49
recall : 0.349
F1 : 0.408
------------------------
=========================================  1
train overdue 10317

------------------------
confusion matrix
[[18709   490]
 [  465   383]]
accuracy : 0.952
precision : 0.439
recall : 0.452
F1 : 0.445
------------------------
=========================================  2
train overdue 13756

------------------------
confusion matrix
[[18588   611]
 [  408   440]]
accuracy : 0.949
precision : 0.419
recall : 0.519
F1 : 0.463
------------------------


In [12]:
# random_state = 0 : 0.461 / 35228 : 0.463 /

### Test Set에서 연체자로 분류된 사람 수

In [15]:
targets = test_data['TARGET']
sum(targets)

97

In [17]:
test_data.to_csv("testset_1012.csv", index=False)